In [1]:
# Test LLM wrapper
from src.llm.OpenAIModel import OpenAIModel
optimizer_config = {"model" : "gpt-4o",
        "temperature" : 1,
        "max_tokens" : None,
        "timeout" : None,
        "max_retries" : 2
        }

optimizer_model = OpenAIModel(name="open_ai_test", config=optimizer_config)
target_config = {"model" : "gpt-4o-mini",
        "temperature" : 1,
        "max_tokens" : None,
        "timeout" : None,
        "max_retries" : 2
        }

target_model = OpenAIModel(name="open_ai_test", config=target_config)

In [2]:
from src.eval.Metric import LLMMetric, Rouge1Metric

general_task_metric_prompt = """You are an evaluator, your job is to rate from 1 to 100 how good the following task was done:
- Task: "{instruction}"
- Expected result: "{target}"
- Predicted: "{predicted}"

Remember just answer the number.
"""
random_llm_metric = LLMMetric("general_task_metric", prompt=general_task_metric_prompt, llm=target_model)

rouge1_metric = Rouge1Metric()


ValueError: could not convert string to float: 'Please provide the values for "instruction," "target," and "predicted" so I can evaluate the task.'

In [4]:
from src.eval.Evaluator import WeightedSumEvaluator

custom_eval = WeightedSumEvaluator(metrics=[random_llm_metric, rouge1_metric], weights={rouge1_metric: 1, "general_task_metric": 10})

In [6]:
from src.eval.TestCase import TestCase

test_set = [
    TestCase(input_vars={"word": "crazy"}, expected_output="sane"),
    TestCase(input_vars={"word": "tall"}, expected_output="short"),
    TestCase(input_vars={"word": "happy"}, expected_output="sad"),
    TestCase(input_vars={"word": "strong"}, expected_output="weak"),
    TestCase(input_vars={"word": "rich"}, expected_output="poor"),
    TestCase(input_vars={"word": "fast"}, expected_output="slow"),
    TestCase(input_vars={"word": "hard"}, expected_output="soft"),
    TestCase(input_vars={"word": "big"}, expected_output="small"),
    TestCase(input_vars={"word": "light"}, expected_output="heavy"),
    TestCase(input_vars={"word": "young"}, expected_output="old"),
    TestCase(input_vars={"word": "noisy"}, expected_output="quiet"),
    TestCase(input_vars={"word": "early"}, expected_output="late")
]

In [8]:
from src.optimization.optimization_job import OptimizationJob, OptimizationJobInput

optimization_job_input = OptimizationJobInput(test_set, "Predict the antonym of the following word: {word}", 3)
optmization_job = OptimizationJob(optimization_job_input, custom_eval, target_model, optimizer_model)

In [9]:
optmization_job.optimize_prompt()

You are an evaluator, your job is to rate from 1 to 100 how good the following task was done:
- Task: "Predict the antonym of the following word: crazy"
- Expected result: "sane"
- Predicted: "The antonym of "crazy" could be "sane.""

Remember just answer the number.

You are an evaluator, your job is to rate from 1 to 100 how good the following task was done:
- Task: "Predict the antonym of the following word: tall"
- Expected result: "short"
- Predicted: "The antonym of "tall" is "short.""

Remember just answer the number.

You are an evaluator, your job is to rate from 1 to 100 how good the following task was done:
- Task: "Predict the antonym of the following word: happy"
- Expected result: "sad"
- Predicted: "The antonym of "happy" is "sad.""

Remember just answer the number.

You are an evaluator, your job is to rate from 1 to 100 how good the following task was done:
- Task: "Predict the antonym of the following word: strong"
- Expected result: "weak"
- Predicted: "The antonym o

KeyboardInterrupt: 

In [ ]:
optmization_job.prompts_tested